In [1]:
import os
import json

# Define the base path based on your directory structure
base_dir = "/Users/xiaoguang_guo@mines.edu/Documents/projects/datasets/imagenet-mini"
val_dir = os.path.join(base_dir, "val")
class_index_file = os.path.join(base_dir, "imagenet_class_index.json")
new_ground_truth_file = os.path.join(base_dir, "imagenet_mini_gt.txt")

# Load the ImageNet class index file
with open(class_index_file, 'r') as f:
    imagenet_class_index = json.load(f)

# Create a WNID-to-index mapping (maps WNIDs to numerical indices, e.g., "n01440764": 0)
wnid_to_index = {v[0]: int(k) for k, v in imagenet_class_index.items()}

# Debug: Print the first few mappings to ensure correctness
print(f"WNID to Index Mapping (First 10): {list(wnid_to_index.items())[:10]}")

# Generate the ground truth labels for the ImageNet Mini validation set
ground_truth_labels = []
image_list = []

# Traverse through the validation directory and generate labels based on folder names
for folder in sorted(os.listdir(val_dir)):
    folder_path = os.path.join(val_dir, folder)
    if os.path.isdir(folder_path) and folder in wnid_to_index:
        class_label = wnid_to_index[folder]  # Get the numerical label for the folder
        for img_name in sorted(os.listdir(folder_path)):
            # Record the image name and its corresponding class index
            image_list.append(img_name)
            ground_truth_labels.append(str(class_label))  # Correct label based on folder WNID

# Write the new ground truth file
with open(new_ground_truth_file, 'w') as f:
    f.write("\n".join(ground_truth_labels))

print(f"New ground truth file created: {new_ground_truth_file}")
print(f"Total labels generated: {len(ground_truth_labels)}")

WNID to Index Mapping (First 10): [('n01440764', 0), ('n01443537', 1), ('n01484850', 2), ('n01491361', 3), ('n01494475', 4), ('n01496331', 5), ('n01498041', 6), ('n01514668', 7), ('n01514859', 8), ('n01518878', 9)]
New ground truth file created: /Users/xiaoguang_guo@mines.edu/Documents/projects/datasets/imagenet-mini/imagenet_mini_gt.txt
Total labels generated: 3923


In [2]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

# Paths to your dataset and necessary files
base_dir = "/Users/xiaoguang_guo@mines.edu/Documents/projects/datasets/imagenet-mini"
val_dir = os.path.join(base_dir, "val")
ground_truth_file = os.path.join(base_dir, "imagenet_mini_gt.txt")
class_index_file = os.path.join(base_dir, "imagenet_class_index.json")

# Load the VGG16 model pre-trained on ImageNet
model = VGG16(weights='imagenet')

# Load ImageNet class index to get the human-readable labels
with open(class_index_file, 'r') as f:
    imagenet_class_index = json.load(f)
    index_to_label = {int(key): value[1] for key, value in imagenet_class_index.items()}

# Load the new ground truth labels file for ImageNet Mini
with open(ground_truth_file, 'r') as f:
    ground_truth_labels = f.readlines()
    ground_truth_labels = [int(label.strip()) for label in ground_truth_labels]

# Debug: Print first few ground truth labels to ensure they are correctly loaded
print(f"First 10 Ground Truth Labels: {ground_truth_labels[:10]}")

# Initialize counters for Top-1 and Top-5 accuracy
top1_correct = 0
top5_correct = 0
total_images = 0

# Set the limit for evaluation
max_images = 50  # Adjust this to evaluate more or fewer images

# Loop through the validation set and evaluate
for folder in sorted(os.listdir(val_dir)):
    folder_path = os.path.join(val_dir, folder)
    if os.path.isdir(folder_path):
        for img_name in sorted(os.listdir(folder_path)):
            if total_images >= max_images:
                break
            img_path = os.path.join(folder_path, img_name)
            
            # Preprocess the image
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # Run inference using VGG16
            preds = model.predict(x)

            # Decode the top-5 predictions
            top5_preds = np.argsort(preds[0])[-5:][::-1]  # Get top-5 indices
            top1_pred = top5_preds[0]  # Top-1 prediction

            # Get the corresponding ground truth label
            ground_truth_label = ground_truth_labels[total_images]
            total_images += 1

            # Debug: Print predictions and ground truth for the first few images
            if total_images <= 10:
                print(f"Image: {img_name}, Top-1 Prediction: {top1_pred}, Ground Truth: {ground_truth_label}")
                print(f"Top-5 Predictions: {top5_preds}")

            # Check if the top-1 prediction matches the ground truth
            if top1_pred == ground_truth_label:
                top1_correct += 1

            # Check if ground truth label is in top-5 predictions
            if ground_truth_label in top5_preds:
                top5_correct += 1

# Calculate accuracy metrics
top1_accuracy = top1_correct / total_images
top5_accuracy = top5_correct / total_images

# Print the evaluation results
print(f"Total Images Evaluated: {total_images}")
print(f"Top-1 Accuracy: {top1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")

First 10 Ground Truth Labels: [0, 0, 0, 1, 1, 1, 2, 2, 2, 2]
1/1 [==============================] - 0s 274ms/step
Image: ILSVRC2012_val_00009111.JPEG, Top-1 Prediction: 0, Ground Truth: 0
Top-5 Predictions: [  0 114 113 758 124]
1/1 [==============================] - 0s 120ms/step
Image: ILSVRC2012_val_00030740.JPEG, Top-1 Prediction: 389, Ground Truth: 0
Top-5 Predictions: [389   0 395 394 391]
1/1 [==============================] - 0s 121ms/step
Image: ILSVRC2012_val_00046252.JPEG, Top-1 Prediction: 0, Ground Truth: 0
Top-5 Predictions: [  0 390 389 997 114]
1/1 [==============================] - 0s 122ms/step
Image: ILSVRC2012_val_00000994.JPEG, Top-1 Prediction: 1, Ground Truth: 1
Top-5 Predictions: [  1 393 392   0  29]
1/1 [==============================] - 0s 122ms/step
Image: ILSVRC2012_val_00002241.JPEG, Top-1 Prediction: 1, Ground Truth: 1
Top-5 Predictions: [  1 971 467 669 880]
1/1 [==============================] - 0s 121ms/step
Image: ILSVRC2012_val_00020822.JPEG, Top-1 P

In [4]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Paths to your dataset and necessary files
base_dir = "/Users/xiaoguang_guo@mines.edu/Documents/projects/datasets/imagenet-mini"
val_dir = os.path.join(base_dir, "val")
ground_truth_file = os.path.join(base_dir, "imagenet_mini_gt.txt")
class_index_file = os.path.join(base_dir, "imagenet_class_index.json")

# Load ImageNet class index to get the human-readable labels
with open(class_index_file, 'r') as f:
    imagenet_class_index = json.load(f)
    index_to_label = {int(key): value[1] for key, value in imagenet_class_index.items()}

# Load the new ground truth labels file for ImageNet Mini
with open(ground_truth_file, 'r') as f:
    ground_truth_labels = f.readlines()
    ground_truth_labels = [int(label.strip()) for label in ground_truth_labels]

# Debug: Print first few ground truth labels to ensure they are correctly loaded
print(f"First 10 Ground Truth Labels: {ground_truth_labels[:10]}")

# Load the TFLite model
tflite_model_path = "VGG16_classification.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details for the interpreter
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize counters for Top-1 and Top-5 accuracy
top1_correct = 0
top5_correct = 0
total_images = 0

# Set the limit for evaluation
max_images = 50  # Adjust this to evaluate more or fewer images

# Loop through the validation set and evaluate
for folder in sorted(os.listdir(val_dir)):
    folder_path = os.path.join(val_dir, folder)
    if os.path.isdir(folder_path):
        for img_name in sorted(os.listdir(folder_path)):
            if total_images >= max_images:
                break
            img_path = os.path.join(folder_path, img_name)
            
            # Preprocess the image
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = x.astype(np.float32)  # Convert to float32 for TFLite
            x = tf.keras.applications.vgg16.preprocess_input(x)

            # Run inference using TFLite interpreter
            interpreter.set_tensor(input_details[0]['index'], x)
            interpreter.invoke()

            # Get predictions from the output tensor
            preds = interpreter.get_tensor(output_details[0]['index'])[0]

            # Decode the top-5 predictions
            top5_preds = np.argsort(preds)[-5:][::-1]  # Get top-5 indices
            top1_pred = top5_preds[0]  # Top-1 prediction

            # Get the corresponding ground truth label
            ground_truth_label = ground_truth_labels[total_images]
            total_images += 1

            # Debug: Print predictions and ground truth for the first few images
            if total_images <= 10:
                print(f"Image: {img_name}, Top-1 Prediction: {top1_pred}, Ground Truth: {ground_truth_label}")
                print(f"Top-5 Predictions: {top5_preds}")

            # Check if the top-1 prediction matches the ground truth
            if top1_pred == ground_truth_label:
                top1_correct += 1

            # Check if ground truth label is in top-5 predictions
            if ground_truth_label in top5_preds:
                top5_correct += 1

# Calculate accuracy metrics
top1_accuracy = top1_correct / total_images
top5_accuracy = top5_correct / total_images

# Print the evaluation results
print(f"Total Images Evaluated: {total_images}")
print(f"Top-1 Accuracy: {top1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")

First 10 Ground Truth Labels: [0, 0, 0, 1, 1, 1, 2, 2, 2, 2]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Image: ILSVRC2012_val_00009111.JPEG, Top-1 Prediction: 0, Ground Truth: 0
Top-5 Predictions: [  0 114 113 758 124]
Image: ILSVRC2012_val_00030740.JPEG, Top-1 Prediction: 389, Ground Truth: 0
Top-5 Predictions: [389   0 395 394 391]
Image: ILSVRC2012_val_00046252.JPEG, Top-1 Prediction: 0, Ground Truth: 0
Top-5 Predictions: [  0 390 389 997 114]
Image: ILSVRC2012_val_00000994.JPEG, Top-1 Prediction: 1, Ground Truth: 1
Top-5 Predictions: [  1 393 392   0  29]
Image: ILSVRC2012_val_00002241.JPEG, Top-1 Prediction: 1, Ground Truth: 1
Top-5 Predictions: [  1 971 467 669 880]
Image: ILSVRC2012_val_00020822.JPEG, Top-1 Prediction: 1, Ground Truth: 1
Top-5 Predictions: [  1 392 973 393 108]
Image: ILSVRC2012_val_00014467.JPEG, Top-1 Prediction: 2, Ground Truth: 2
Top-5 Predictions: [  2   6 801 433  33]
Image: ILSVRC2012_val_00019409.JPEG, Top-1 Prediction: 2, Ground Truth: 2
Top-5 Predictions: [  2   3   4  33 148]
Image: ILSVRC2012_val_00027514.JPEG, Top-1 Prediction: 2, Ground Truth: 2
Top-

In [6]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.applications import VGG16

# Step 1: Load the Pre-trained VGG16 Model
base_model = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Step 2: Pruning the Model with 70% Sparsity
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0, final_sparsity=0.7, begin_step=0, end_step=1000)
}

# Apply pruning to the entire model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(base_model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
pruned_model.summary()

# Step 3: Strip pruning wrappers from the model
stripped_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Save the stripped model without pruning wrappers
stripped_model_filename = "vgg16_pruned_stripped.h5"
stripped_model.save(stripped_model_filename)
print(f"Stripped pruned model saved as {stripped_model_filename}")

# Step 4: Convert the Stripped Model to a TFLite Model with Quantization
# Load the stripped model again to ensure it's ready for conversion
loaded_model = tf.keras.models.load_model(stripped_model_filename, compile=False)

# Define a representative dataset function for int8 quantization
def representative_data_gen():
    for _ in range(100):
        data = np.random.rand(1, 224, 224, 3).astype(np.float32)
        yield [data]

# Convert the stripped model to TFLite format with int8 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # Quantize inputs to int8
converter.inference_output_type = tf.int8  # Quantize outputs to int8

# Convert the stripped model to a TFLite quantized model
tflite_quantized_model = converter.convert()

# Save the quantized TFLite model
tflite_filename = "vgg16_pruned_stripped_quantized_int8.tflite"
with open(tflite_filename, "wb") as f:
    f.write(tflite_quantized_model)

print(f"Saved the pruned, stripped, and quantized VGG16 model as {tflite_filename}")

# Step 5: Print Model Size Reduction
stripped_size = os.path.getsize(stripped_model_filename) / (1024 * 1024)
quantized_size = os.path.getsize(tflite_filename) / (1024 * 1024)
print(f"Stripped Pruned Model Size: {stripped_size:.2f} MB")
print(f"Quantized Model Size: {quantized_size:.2f} MB")

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 prune_low_magnitude_block1  (None, 224, 224, 64)      3522      
 _conv1 (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_block1  (None, 224, 224, 64)      73794     
 _conv2 (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_block1  (None, 112, 112, 64)      1         
 _pool (PruneLowMagnitude)                                       
                                                                 
 prune_low_magnitude_block2  (None, 112, 112, 128)     147586    
 _conv1 (PruneLowMagnitude)                                  

Stripped pruned model saved as vgg16_pruned_stripped.h5
INFO:tensorflow:Assets written to: /var/folders/hb/fpmv_swd6pb2nt84zqhq62fh0000gq/T/tmpkahogt2a/assets


INFO:tensorflow:Assets written to: /var/folders/hb/fpmv_swd6pb2nt84zqhq62fh0000gq/T/tmpkahogt2a/assets
/Users/xiaoguang_guo@mines.edu/anaconda3/envs/mcu/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-10-10 21:13:11.303895: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-10-10 21:13:11.303908: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-10-10 21:13:11.304041: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/hb/fpmv_swd6pb2nt84zqhq62fh0000gq/T/tmpkahogt2a
2024-10-10 21:13:11.305536: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-10-10 21:13:11.305541: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/hb/fpmv_s

Saved the pruned, stripped, and quantized VGG16 model as vgg16_pruned_stripped_quantized_int8.tflite
Stripped Pruned Model Size: 527.86 MB
Quantized Model Size: 132.09 MB


In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Paths to your dataset and necessary files
base_dir = "/Users/xiaoguang_guo@mines.edu/Documents/projects/datasets/imagenet-mini"
val_dir = os.path.join(base_dir, "val")
ground_truth_file = os.path.join(base_dir, "imagenet_mini_gt.txt")

# Load the TFLite model
tflite_model_path = "vgg16_pruned_stripped_quantized_int8.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print the expected input data type and shape
print(f"Expected Input Data Type: {input_details[0]['dtype']}")  # Should print `int8`
print(f"Expected Input Shape: {input_details[0]['shape']}")     # Should print `[1, 224, 224, 3]`

# Load the ground truth labels
with open(ground_truth_file, 'r') as f:
    ground_truth_labels = f.readlines()
    ground_truth_labels = [int(label.strip()) for label in ground_truth_labels]

# Debug: Print first few ground truth labels to ensure they are correctly loaded
print(f"First 10 Ground Truth Labels: {ground_truth_labels[:10]}")

# Initialize counters for Top-1 and Top-5 accuracy
top1_correct = 0
top5_correct = 0
total_images = 0

# Set the limit for evaluation
max_images = 50  # Adjust this to evaluate more or fewer images

# Loop through the validation set and evaluate
for folder in sorted(os.listdir(val_dir)):
    folder_path = os.path.join(val_dir, folder)
    if os.path.isdir(folder_path):
        for img_name in sorted(os.listdir(folder_path)):
            if total_images >= max_images:
                break
            img_path = os.path.join(folder_path, img_name)
            
            # Preprocess the image to match the TFLite model's input requirements (INT8)
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)

            # Scale the input image to the appropriate range expected by the quantized model
            # The quantized model expects input in range [-128, 127] when using INT8 precision
            # Adjust scaling to match this expectation
            x = (x - 128).astype(np.int8)  # Scale the image and convert to int8

            # Expand dimensions to match model input and set tensor
            x = np.expand_dims(x, axis=0)
            
            # Assign the preprocessed image to the TFLite model's input tensor
            interpreter.set_tensor(input_details[0]['index'], x)

            # Run inference using the TFLite model
            interpreter.invoke()

            # Retrieve the predictions from the output tensor
            preds = interpreter.get_tensor(output_details[0]['index'])[0]

            # Decode the top-5 predictions
            top5_preds = np.argsort(preds)[-5:][::-1]  # Get top-5 indices
            top1_pred = top5_preds[0]  # Top-1 prediction

            # Get the corresponding ground truth label
            ground_truth_label = ground_truth_labels[total_images]
            total_images += 1

            # Debug: Print predictions and ground truth for the first few images
            if total_images <= 10:
                print(f"Image: {img_name}, Top-1 Prediction: {top1_pred}, Ground Truth: {ground_truth_label}")
                print(f"Top-5 Predictions: {top5_preds}")

            # Check if the top-1 prediction matches the ground truth
            if top1_pred == ground_truth_label:
                top1_correct += 1

            # Check if ground truth label is in top-5 predictions
            if ground_truth_label in top5_preds:
                top5_correct += 1

# Calculate accuracy metrics
top1_accuracy = top1_correct / total_images
top5_accuracy = top5_correct / total_images

# Print the evaluation results
print(f"Total Images Evaluated: {total_images}")
print(f"Top-1 Accuracy: {top1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")

Expected Input Data Type: <class 'numpy.int8'>
Expected Input Shape: [  1 224 224   3]
First 10 Ground Truth Labels: [0, 0, 0, 1, 1, 1, 2, 2, 2, 2]
Image: ILSVRC2012_val_00009111.JPEG, Top-1 Prediction: 999, Ground Truth: 0
Top-5 Predictions: [999 431 669 794 700]
Image: ILSVRC2012_val_00030740.JPEG, Top-1 Prediction: 999, Ground Truth: 0
Top-5 Predictions: [999 700 669 794 431]
Image: ILSVRC2012_val_00046252.JPEG, Top-1 Prediction: 794, Ground Truth: 0
Top-5 Predictions: [794 669 999   6 419]
Image: ILSVRC2012_val_00000994.JPEG, Top-1 Prediction: 669, Ground Truth: 1
Top-5 Predictions: [669 999 794  29   6]
Image: ILSVRC2012_val_00002241.JPEG, Top-1 Prediction: 669, Ground Truth: 1
Top-5 Predictions: [669 794 999 712  58]
Image: ILSVRC2012_val_00020822.JPEG, Top-1 Prediction: 669, Ground Truth: 1
Top-5 Predictions: [669 794 999 431  29]
Image: ILSVRC2012_val_00014467.JPEG, Top-1 Prediction: 999, Ground Truth: 2
Top-5 Predictions: [999 669 794 700 549]
Image: ILSVRC2012_val_00019409.JP

In [16]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Prune the model
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0, final_sparsity=0.7, begin_step=0, end_step=100)
}
pruned_model = sparsity.prune_low_magnitude(vgg16_model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the pruned model
pruned_model_filename = "vgg16_pruned.h5"
pruned_model.save(pruned_model_filename)
print(f"Pruned model saved as {pruned_model_filename}")

/Users/xiaoguang_guo@mines.edu/anaconda3/envs/mcu/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Pruned model saved as vgg16_pruned.h5


In [19]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude

# Define a custom object scope to correctly register the PruneLowMagnitude layer
with tf.keras.utils.custom_object_scope({'PruneLowMagnitude': prune_low_magnitude}):
    pruned_model = tf.keras.models.load_model(pruned_model_filename, compile=False)

# Continue with the quantization and TFLite conversion process
def representative_data_gen():
    for _ in range(100):
        yield [np.random.rand(1, 224, 224, 3).astype(np.float32)]

# Convert the pruned model to TFLite format with int8 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quantized_model = converter.convert()

# Save the quantized TFLite model
tflite_model_filename = "vgg16_pruned_quantized_fixed.tflite"
with open(tflite_model_filename, "wb") as f:
    f.write(tflite_quantized_model)

print(f"Quantized model saved as {tflite_model_filename}")

TypeError: prune_low_magnitude() missing 1 required positional argument: 'to_prune'